In [ ]:
import optuna
import pandas as pd

from src.algorithms.tabu_move import solve_tsp
from src.utils.tsp_loader import load_tsp_matrix
from src.utils.result_saver import save_experiment_results

In [ ]:
# Konfiguracja eksperymentu
INSTANCES = [
    'Dane_TSP_48.xlsx',
    'Dane_TSP_76.xlsx',
    'Dane_TSP_127.xlsx'
]

# Liczba prób optymalizacji dla każdej instancji
N_TRIALS = 100

In [3]:
def objective(trial, distance_matrix):
    """
    Funkcja celu dla Optuny.
    Optuna dobiera parametry, my uruchamiamy algorytm i zwracamy wynik (koszt).
    """
    # Definiujemy zakresy, z których Optuna może losować wartości.
    # Używamy parametru 'step', aby zmniejszyć liczbę możliwych kombinacji, by nie gubiła się w niepotrzebnych kombinacjach.
    
    max_iter = trial.suggest_int("max_iter", 5_000, 100_000, step=5_000)
    stop_no_improve = trial.suggest_int("stop_no_improve", 500, 10_000, step=500)
    
    tabu_tenure = trial.suggest_int("tabu_tenure", 5, 100, step=5)
    n_neighbors = trial.suggest_int("n_neighbors", 10, 150, step=10)
    
    # Zmienne kategoryczne
    neighborhood_type = trial.suggest_categorical("neighborhood_type", ["swap", "insert", "two_opt"])
    
    # Parametry przekazywane do funkcji
    params = {
        "max_iter": max_iter,
        "stop_no_improve": stop_no_improve,
        "tabu_tenure": tabu_tenure,
        "n_neighbors": n_neighbors,
        "neighborhood_type": neighborhood_type
    }
    
    # Uruchomienie Algorytmu Tabu Search
    # solve_tsp zwraca m.in. najlepszą trasę (best_route) i jej koszt (best_cost)
    best_route, best_cost, _, _ = solve_tsp(distance_matrix, params)
    
    # Zapisanie najlepszej trasy dla tego przebiegu, by móc potem dodać do wyników csv
    # Optuna pozwala zapisać dodatkowe atrybuty (user_attrs) dla każdego triala.
    # Konwertujemy trasę na czytelny string: "0-5-2-10..." tak jak dla reszty algorytmów
    route_str = "-".join(map(str, best_route))
    trial.set_user_attr("min_route", route_str)
    
    # Zwracamy wartość jest rozpatrywana przez Optuna jako wartość do minimalizacji
    # Zwracamy koszt
    return best_cost

In [4]:
experiment_results = []

for instance_file in INSTANCES:
    print(f"\nOptymalizacja dla instancji: {instance_file}")
    
    distance_matrix = load_tsp_matrix(instance_file)
    
    # Ustawienie ziarna (seed) dla losowości
    # Dzięki temu uruchamiając kod ponownie, uzyskamy te same wyniki
    sampler = optuna.samplers.TPESampler(seed=42)

    # direction="minimize" oznacza, że szukamy jak najmniejszego kosztu
    study = optuna.create_study(direction="minimize", sampler=sampler)
    
    # Pierwsze parametry do sprawdzenia przez Optune
    study.enqueue_trial({
        "max_iter": 5_000,
        "stop_no_improve": 500,
        "tabu_tenure": 10,
        "n_neighbors": 30,
        "neighborhood_type": "two_opt"
    })

    # Uruchomienie optymalizacji
    study.optimize(lambda trial: objective(trial, distance_matrix), n_trials=N_TRIALS)
    
    print(f"Najlepsze parametry dla {instance_file}: {study.best_params}")
    print(f"Najlepszy koszt dla {instance_file}: {study.best_value}")
    
    # Pobranie wyników wszystkich prób (triali) do DataFrame
    df_trials = study.trials_dataframe()
    
    # Przetworzenie wyników do formatu zgodnego z resztą projektu
    for _, row in df_trials.iterrows():
        record = {
            "instance": instance_file,
            "max_iter": row.get("params_max_iter"),
            "tabu_tenure": row.get("params_tabu_tenure"),
            "n_neighbors": row.get("params_n_neighbors"),
            "neighborhood_type": row.get("params_neighborhood_type"),
            "stop_no_improve": row.get("params_stop_no_improve"),
            # Wynik (koszt)
            "min_cost": row.get("value"),
            "min_route": row.get("user_attrs_min_route"),
            "state": row.get("state"),
            "trial_number": row.get("number")
        }
        experiment_results.append(record)


Optymalizacja dla instancji: Dane_TSP_48.xlsx


[I 2025-12-16 10:54:30,542] A new study created in memory with name: no-name-12afab7d-82af-4b55-a184-c6edda404d81
[I 2025-12-16 10:54:31,471] Trial 0 finished with value: 10829.0 and parameters: {'max_iter': 5000, 'stop_no_improve': 500, 'tabu_tenure': 10, 'n_neighbors': 30, 'neighborhood_type': 'two_opt'}. Best is trial 0 with value: 10829.0.
[I 2025-12-16 10:54:43,580] Trial 1 finished with value: 11516.0 and parameters: {'max_iter': 40000, 'stop_no_improve': 10000, 'tabu_tenure': 75, 'n_neighbors': 90, 'neighborhood_type': 'swap'}. Best is trial 0 with value: 10829.0.
[I 2025-12-16 10:54:44,425] Trial 2 finished with value: 16423.0 and parameters: {'max_iter': 90000, 'stop_no_improve': 6500, 'tabu_tenure': 75, 'n_neighbors': 10, 'neighborhood_type': 'swap'}. Best is trial 0 with value: 10829.0.
[I 2025-12-16 10:54:47,013] Trial 3 finished with value: 10217.0 and parameters: {'max_iter': 20000, 'stop_no_improve': 2000, 'tabu_tenure': 35, 'n_neighbors': 80, 'neighborhood_type': 'two_o

Najlepsze parametry dla Dane_TSP_48.xlsx: {'max_iter': 20000, 'stop_no_improve': 2000, 'tabu_tenure': 35, 'n_neighbors': 80, 'neighborhood_type': 'two_opt'}
Najlepszy koszt dla Dane_TSP_48.xlsx: 10217.0

Optymalizacja dla instancji: Dane_TSP_76.xlsx


[I 2025-12-16 10:54:51,041] Trial 0 finished with value: 114698.06013029454 and parameters: {'max_iter': 5000, 'stop_no_improve': 500, 'tabu_tenure': 10, 'n_neighbors': 30, 'neighborhood_type': 'two_opt'}. Best is trial 0 with value: 114698.06013029454.
[I 2025-12-16 10:55:29,218] Trial 1 finished with value: 123941.56896671056 and parameters: {'max_iter': 40000, 'stop_no_improve': 10000, 'tabu_tenure': 75, 'n_neighbors': 90, 'neighborhood_type': 'swap'}. Best is trial 0 with value: 114698.06013029454.
[I 2025-12-16 10:55:31,693] Trial 2 finished with value: 235381.92207127623 and parameters: {'max_iter': 90000, 'stop_no_improve': 6500, 'tabu_tenure': 75, 'n_neighbors': 10, 'neighborhood_type': 'swap'}. Best is trial 0 with value: 114698.06013029454.
[I 2025-12-16 10:55:51,162] Trial 3 finished with value: 98983.7206949425 and parameters: {'max_iter': 20000, 'stop_no_improve': 2000, 'tabu_tenure': 35, 'n_neighbors': 80, 'neighborhood_type': 'two_opt'}. Best is trial 3 with value: 98983

Najlepsze parametry dla Dane_TSP_76.xlsx: {'max_iter': 20000, 'stop_no_improve': 2000, 'tabu_tenure': 35, 'n_neighbors': 80, 'neighborhood_type': 'two_opt'}
Najlepszy koszt dla Dane_TSP_76.xlsx: 98983.7206949425

Optymalizacja dla instancji: Dane_TSP_127.xlsx


[I 2025-12-16 10:56:14,577] A new study created in memory with name: no-name-4472da0b-f5e4-4bb6-8349-b3930397e7bf
[I 2025-12-16 10:56:17,062] Trial 0 finished with value: 136586.54716379955 and parameters: {'max_iter': 5000, 'stop_no_improve': 500, 'tabu_tenure': 10, 'n_neighbors': 30, 'neighborhood_type': 'two_opt'}. Best is trial 0 with value: 136586.54716379955.
[I 2025-12-16 10:57:34,939] Trial 1 finished with value: 138020.89585906285 and parameters: {'max_iter': 40000, 'stop_no_improve': 10000, 'tabu_tenure': 75, 'n_neighbors': 90, 'neighborhood_type': 'swap'}. Best is trial 0 with value: 136586.54716379955.
[I 2025-12-16 10:57:41,347] Trial 2 finished with value: 247296.6781156928 and parameters: {'max_iter': 90000, 'stop_no_improve': 6500, 'tabu_tenure': 75, 'n_neighbors': 10, 'neighborhood_type': 'swap'}. Best is trial 0 with value: 136586.54716379955.
[I 2025-12-16 10:57:49,397] Trial 3 finished with value: 113313.85047050464 and parameters: {'max_iter': 20000, 'stop_no_impro

Najlepsze parametry dla Dane_TSP_127.xlsx: {'max_iter': 20000, 'stop_no_improve': 2000, 'tabu_tenure': 35, 'n_neighbors': 80, 'neighborhood_type': 'two_opt'}
Najlepszy koszt dla Dane_TSP_127.xlsx: 113313.85047050464


In [ ]:
# Zapisanie wyników do pliku CSV
if experiment_results:
    df_final = pd.DataFrame(experiment_results)
    
    # Sortowanie wyników: najpierw po instancji, potem po numerze triala (0, 1, 2...)
    df_final = df_final.sort_values(by=["instance", "trial_number"])
    
    # Funkcja save_experiment_results automatycznie tworzy nazwę pliku z datą
    save_experiment_results(df_final, time_seconds=0, subfolder="TS_Optuna", sort_by_cost=False)
    
    print("\nWyniki zostały zapisane pomyślnie w folderze project/results/TS_Optuna/")
    print(df_final.head())


Podsumowanie (pierwsze 20 wierszy):
         instance  max_iter  tabu_tenure  n_neighbors neighborhood_type  stop_no_improve      min_cost                                                                                                                                                                                                                                                                                                                                                                                                         min_route    state  trial_number
Dane_TSP_127.xlsx      5000           10           30           two_opt              500 136586.547164 46-48-94-112-111-107-127-93-95-123-97-98-0-101-102-82-83-126-81-84-117-18-78-79-80-25-29-32-122-28-33-38-26-43-34-42-39-40-30-31-14-41-27-12-16-105-106-9-3-114-77-68-74-73-76-75-71-70-69-63-119-96-109-88-87-86-85-110-104-125-89-92-99-65-55-66-124-56-57-50-115-10-6-11-120-13-5-52-58-113-64-100-91-62-61-90-60-59-116-67-8-72-17-21-19